In [ ]:
import numpy as np
import cv2

In [ ]:
def get_gabor(images):
    filters = []
    ksize = 9
    sigma = 0.1
    gamma = 0.5
    phi = 0
    
    # define the range for theta and nu
    for theta in np.arange(0, np.pi, np.pi / 8):
        for nu in np.arange(0, 6*np.pi/4, np.pi / 4):
            kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, nu, gamma, phi, ktype=cv2.CV_32F)
            kern /= 1.5*kern.sum()
            filters.append(kern)
    
    # function to convolve the image with the filters
    def process(img, filters):
        accum = np.zeros_like(img)
        for kern in filters:
            fimg = cv2.filter2D(img, cv2.CV_8UC3, kern)
            np.maximum(accum, fimg, accum)
            return accum
        
    
    def get_image_gabor(image):
        # Local Binary Pattern
        f = []

        # calculating the local energy for each convolved image
        for j in range(40):
            res = process(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY), filters[j])
            res = np.array(res)
            f.append(np.sum(np.multiply(res, res)))

        # calculating the mean amplitude for each convolved image
        for j in range(40):
            res = process(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY), filters[j])
            res = np.array(res)
            f.append(np.sum(abs(res)))
        return f
    
    # Gabor descriptor for all images
    features = []
    for i, img in enumerate(images):
        f = get_image_gabor(img)
        features.append(f)
    
    result = np.array(features)
    
    return result